# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
# from azureml.core.workspace import Workspace
# from azureml.core import Environment

In [3]:
# myenv = Environment.from_conda_specification(name="myenv", file_path="envFile.yml")

In [2]:
# !pip uninstall azureml-train-automl-runtime 

In [3]:
# import azureml.automl.runtime
# import azureml.core
# import azureml.dataprep
# import azureml.dataprep.native
# import azureml.dataprep.rslex
# import azureml.dataset.runtime
# import azureml.defaults
# import azureml.interpret
# import azureml.pipeline.core
# import azureml.telemetry
# import azureml.train.automl.client
# import azureml.train.automl.runtime

In [4]:
# import azureml.train.automl
# import azureml.automl.core
# import importlib
# importlib.reload(azureml.train.automl)
# importlib.reload(azureml.automl.core)
# importlib.reload(azureml-automl-core==1.20.0)


# importlib.reload(azureml-automl-runtime==1.20.0)
# importlib.reload(azureml-core==1.20.0)
# importlib.reload(azureml-dataprep==2.7.2
# importlib.reload(azureml-dataprep-native==27.0).0)
# importlib.reload(azureml-dataprep-rslex==1.5.0)
# importlib.reload(azureml-dataset-runtime==1.20.0)
# importlib.reload(azureml-defaults==1.20.0)
# importlib.reload(azureml-interpret==1.20.0)
# importlib.reload(azureml-pipeline-core==1.20.0)
# importlib.reload(azureml-telemetry==1.20.0)
# importlib.reload(azureml-train-automl-client==1.20.0)

In [5]:
#!pip install --upgrade azureml-automl-core==1.20.0

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core import Workspace, Dataset

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


SDK version: 1.20.0


In [5]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-experiment'
project_folder = './pipeline-project'
experiment=Experiment(ws, experiment_name)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-135691
aml-quickstarts-135691
southcentralus
f5091c60-1c3c-430f-8d81-d802f6bf2414


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [35]:
dataset = Dataset.get_by_name(workspace=ws, name='online_shoppers_intention')
df = dataset.to_pandas_dataframe()

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "project3"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [8]:
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Revenue",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output=True)

Running on remote.
No run_configuration provided, running on project3 with default configuration
Running on remote compute: project3
Parent Run ID: AutoML_918668b3-b496-4eac-a938-568ba54b9fbc

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

******************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------

{'runId': 'AutoML_918668b3-b496-4eac-a938-568ba54b9fbc',
 'target': 'project3',
 'status': 'Completed',
 'startTimeUtc': '2021-01-22T11:19:16.933414Z',
 'endTimeUtc': '2021-01-22T11:45:52.495796Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'project3',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment","subscription_id":"f5091c60-1c3c-430f-8d81-d802f6bf2414","resource_group":"aml-quickstarts-135691","workspace_name":"quick-starts-ws-135691","region":"southcentralus","compute_target":"project3","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"fe

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
automl_best_run, fitted_model = remote_run.get_output()
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               reg_lambda=0.20833333333333334,
                                                                                               scale_pos_weight=1,
                                                                                               seed=None,
                                            

In [16]:
#TODO: Save the best model
automl_model = automl_best_run.register_model(model_name='classification_automl',model_path='outputs/')

In [17]:
# env = automl_best_run.get_environment()
# automl_best_run.download_files?

In [18]:
# automl metrics
automl_metrics = automl_best_run.get_metrics()
for metric in automl_metrics:
    print('Metric name : {} = {}'.format(metric,automl_metrics[metric]))

Metric name : average_precision_score_micro = 0.9704458260247005
Metric name : AUC_weighted = 0.9339241775471206
Metric name : f1_score_micro = 0.9082725060827249
Metric name : f1_score_macro = 0.8109262548598518
Metric name : matthews_correlation = 0.6265814674326827
Metric name : accuracy = 0.9082725060827249
Metric name : f1_score_weighted = 0.9044997286488763
Metric name : recall_score_micro = 0.9082725060827249
Metric name : average_precision_score_macro = 0.8695532524312992
Metric name : precision_score_micro = 0.9082725060827249
Metric name : recall_score_weighted = 0.9082725060827249
Metric name : recall_score_macro = 0.7889270392450879
Metric name : norm_macro_recall = 0.5778540784901761
Metric name : balanced_accuracy = 0.7889270392450879
Metric name : log_loss = 0.2415734760282476
Metric name : weighted_accuracy = 0.9506636670144618
Metric name : AUC_micro = 0.9695363118459714
Metric name : precision_score_weighted = 0.9032187214790316
Metric name : average_precision_score_w

In [55]:
# automl metrics
#automl_metrics = automl_best_run.get_metrics()
print('Metric name : accuracy = {}'.format(automl_metrics['accuracy']))

Metric name : accuracy = 0.9082725060827249


In [50]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    print(model.properties)

classification_hyperdrive version: 1
{}
classification_automl version: 1
{}


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
# Download scoring file 
automl_best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

# Download environment file
automl_best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')
automl_best_run.download_file('outputs/env_dependencies.json', 'envDependencies.json')


TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model
#automl_best_run.get_environment()

In [33]:
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=automl_best_run.get_environment())

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4,enable_app_insights=True)
service = Model.deploy(ws, "shoppingintension", [automl_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://0e87fbaf-14c4-45a2-8d6c-8f9b5ea7de00.southcentralus.azurecontainer.io/score
http://0e87fbaf-14c4-45a2-8d6c-8f9b5ea7de00.southcentralus.azurecontainer.io/swagger.json


In [34]:
print(service.get_logs())

2021-01-22T12:50:56,251155700+00:00 - iot-server/run 
2021-01-22T12:50:56,277198200+00:00 - gunicorn/run 
2021-01-22T12:50:56,278819400+00:00 - rsyslog/run 
2021-01-22T12:50:56,293920500+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_23df5d07c5e8388dac3ecfdf573dafb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

TODO: In the cell below, print the logs of the web service and delete the service

In [30]:
Model.get_model_path('classification_automl',_workspace=ws)

'azureml-models/classification_automl/1/outputs'

In [37]:
import json
#Creating Test data
#Created test data from the original data bytaking first 10 rows
test_df = df.sample(10) 
test_df_label = test_df.pop('Revenue')

test_data = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_data)

{"data": [{"Administrative": 0.0, "Administrative_Duration": 0.0, "Informational": 0.0, "Informational_Duration": 0.0, "ProductRelated": 3.0, "ProductRelated_Duration": 58.5, "BounceRates": 0.0, "ExitRates": 0.066666667, "PageValues": 0.0, "SpecialDay": 0.0, "Month": "Dec", "OperatingSystems": 2, "Browser": 4, "Region": 1, "TrafficType": 2, "VisitorType": "Returning_Visitor", "Weekend": false}, {"Administrative": 1.0, "Administrative_Duration": 0.0, "Informational": 0.0, "Informational_Duration": 0.0, "ProductRelated": 10.0, "ProductRelated_Duration": 304.0, "BounceRates": 0.0, "ExitRates": 0.02, "PageValues": 0.0, "SpecialDay": 0.0, "Month": "May", "OperatingSystems": 3, "Browser": 2, "Region": 4, "TrafficType": 4, "VisitorType": "Returning_Visitor", "Weekend": true}, {"Administrative": 4.0, "Administrative_Duration": 136.0, "Informational": 0.0, "Informational_Duration": 0.0, "ProductRelated": 31.0, "ProductRelated_Duration": 1141.833333, "BounceRates": 0.0, "ExitRates": 0.006060606,

In [38]:
# We will send POST request to the deployed service for that we will import the request module
import requests 

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_data, headers=headers)

In [46]:
response.text

'"{\\"result\\": [false, false, true, false, false, false, false, false, false, false]}"'

In [44]:
test_df_label.to_list()

[False, False, True, False, False, False, False, False, False, False]

In [32]:
service.delete()

In [11]:
# pip install --upgrade azureml-automl-core==1.20.0

# pip install --upgrade
# pip install --upgrade azureml-automl-runtime==1.20.0
# pip install --upgrade azureml-core==1.20.0
# pip install --upgrade azureml-dataprep==2.7.2
# c
# cc
# pip install --upgrade azureml-dataset-runtime==1.20.0
# pip install --upgrade azureml-defaults==1.20.0
# pip install --upgrade azureml-interpret==1.20.0
# pip install --upgrade azureml-pipeline-core==1.20.0
# pip install --upgrade azureml-telemetry==1.20.0
# pip install --upgrade azureml-train-automl-client==1.20.0
# pip install --upgrade azureml-train-automl-runtime==1.20.0
